<a href="https://colab.research.google.com/github/youssouf2230/VisaBot_AgentAi_LLamaIndex_RAG/blob/main/Assistant_Visa_Intelligent_(VisaBot_AI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assistant** *Visa Intelligent* (VisaBot AI)
**Objectif** :

Créer un assistant intelligent capable de répondre aux questions des utilisateurs sur les demandes de visa, en utilisant :

RAG (pour interroger des documents PDF / pages web officielles)

Agent AI (pour gérer des dialogues complexes)

Tools personnalisés (comme un outil de vérification de documents ou de localisation d’ambassade)

In [ ]:
# installation des bibliothèques
!pip install --upgrade pip
!pip install llama-index llama-index-llms-mistralai llama-index-embeddings-huggingface langchain sentence-transformers

# réupère clé de mistral

In [ ]:
from google.colab import userdata
mistral_key_api=userdata.get('visaBot_key')

1. **Données**

In [ ]:
import requests
import os
from llama_index.readers.file import PDFReader

# Liste des URLs
urls = [
    "https://static.tlscontact.com/media/ma/cas/be/2019_08_liste_harmonisee_documents_a_soumettre_pour_un_visa_de_court_sejour.pdf",
    "https://ma.consulfrance.org/IMG/pdf/pieces_a_fournir_etudiants_majeur.pdf?2453/cc842d3a1bfe1380d73c6608ca350250691199fd",
]

# Dossier de sauvegarde
save_dir = "pdf_visa_docs"
os.makedirs(save_dir, exist_ok=True)

file_paths = []
for i, url in enumerate(urls):
    response = requests.get(url)

    #  Vérifie que le fichier est bien un PDF
    content_type = response.headers.get("Content-Type", "")
    if "application/pdf" in content_type:
        filename = f"visa_doc_{i+1}.pdf"
        filepath = os.path.join(save_dir, filename)
        with open(filepath, "wb") as f:
            f.write(response.content)
        file_paths.append(filepath)
        print(f"Téléchargé : {filename}")
    else:
        print(f"Mauvais contenu (pas un PDF) pour l’URL : {url}")

# Lire les fichiers valides
loader = PDFReader()
documents = []
for path in file_paths:
    docs = loader.load_data(file=path)
    documents.extend(docs)

# Affichage
print(f"\n {len(documents)} document(s) PDF valides chargés.")
print(documents[0].text[:500])


Téléchargé : visa_doc_1.pdf
Mauvais contenu (pas un PDF) pour l’URL : https://ma.consulfrance.org/IMG/pdf/pieces_a_fournir_etudiants_majeur.pdf?2453/cc842d3a1bfe1380d73c6608ca350250691199fd

 8 document(s) PDF valides chargés.
Liste des documents justificatifs devant être produits par les demandeurs de visa de 
court séjour au Maroc  
 
I. Documents à présenter par tous les demandeurs de visa  
  
1. Preuve du moyen de transport  
- réservation d'un billet aller-retour (avion, autobus ou bateau), selon le cas.  
2. Preuve d'hébergement  
- réservation(s) d'hôtel, ou  
- preuve que le demandeur de visa loue ou possède des biens immobiliers dans l’État 
membre de destination, ou  
- confirmation de l’hébergement chez un


In [ ]:
!pip install playwright
!playwright install

173.7 MiB [] 0% 0.0s173.7 MiB [] 0% 4.7s173.7 MiB [] 0% 3.3s173.7 MiB [] 1% 2.9s173.7 MiB [] 2% 3.0s173.7 MiB [] 2% 2.7s173.7 MiB [] 3% 2.8s173.7 MiB [] 3% 2.7s173.7 MiB [] 4% 2.8s173.7 MiB [] 5% 2.7s173.7 MiB [] 5% 2.8s173.7 MiB [] 6% 2.8s173.7 MiB [] 7% 2.7s173.7 MiB [] 8% 2.6s173.7 MiB [] 9% 2.6s173.7 MiB [] 9% 2.7s173.7 MiB [] 10% 2.5s173.7 MiB [] 11% 2.4s173.7 MiB [] 12% 2.3s173.7 MiB [] 13% 2.3s173.7 MiB [] 14% 2.2s173.7 MiB [] 15% 2.2s173.7 MiB [] 16% 2.1s173.7 MiB [] 17% 2.0s173.7 MiB [] 18% 1.9s173.7 MiB [] 19% 1.9s173.7 MiB [] 20% 1.8s173.7 MiB [] 21% 1.8s173.7 MiB [] 22% 1.8s173.7 MiB [] 23% 1.7s173.7 MiB [] 24% 1.7s173.7 MiB [] 25% 1.7s173.7 MiB [] 26% 1.7s173.7 MiB [] 27% 1.7s173.7 MiB [] 28% 1.6s173.7 MiB [] 30% 1.6s173.7 MiB [] 31% 1.5s173.7 MiB [] 32% 1.5s173.7 MiB [] 33% 1.5s173.7 MiB [] 34% 1.4s173.7 MiB [] 35% 1.4s173.7 MiB [] 36% 1.3s173.7 MiB [] 37% 1.3s173.7 MiB [] 39% 1.2s173.7 MiB [] 40% 1.2s173.7 MiB [] 41% 1.2s173.7 MiB [] 42% 1.1s173.7 MiB [] 43% 1.1s173.7 Mi

In [ ]:
import asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from llama_index.core.schema import Document

async def get_rendered_text_from_url_async(url: str) -> str:
    """Utilise Playwright async pour extraire le texte visible d'une page web."""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)
        await page.wait_for_timeout(5000)  # Laisse le JS s'exécuter

        html = await page.content()
        await browser.close()

        # Extraire texte lisible
        soup = BeautifulSoup(html, "html.parser")
        text = soup.get_text(separator="\n", strip=True)
        return text

async def load_all_urls_to_documents(urls):
    documents = []
    for url in urls:
        print(f"Chargement de : {url}")
        try:
            text = await get_rendered_text_from_url_async(url)
            doc = Document(text=text, metadata={"source": url})
            documents.append(doc)
        except Exception as e:
            print(f"Erreur lors du chargement de {url}: {e}")
    return documents

# Liste des URLs
urls_web = [
    "https://www.service-public.fr/particuliers/vosdroits/F16162",
    "https://france-visas.gouv.fr/fr/web/france-visas/etudiants",
    "https://www.skyscanner.fr/transport/vols/ma/fr/?adultsv2=1"
]

# Lancer la tâche Playwright async
web_docs = asyncio.run(load_all_urls_to_documents(urls_web))
# Afficher un extrait des textes récupérés
for i, doc in enumerate(web_docs):
    print(f"\n--- Extrait de la page {i+1} ({doc.metadata['source']}) ---\n")
    print(doc.text[:1500])  # Affiche les 1500 premiers caractères

# Ajouter aux documents
documents.extend(web_docs)
print(f"{len(web_docs)} page(s) web rendue(s) ajoutée(s).")
print(f"Total documents pour le RAG : {len(documents)}")


Chargement de : https://www.service-public.fr/particuliers/vosdroits/F16162
Chargement de : https://france-visas.gouv.fr/fr/web/france-visas/etudiants
Chargement de : https://www.skyscanner.fr/transport/vols/ma/fr/?adultsv2=1

--- Extrait de la page 1 (https://www.service-public.fr/particuliers/vosdroits/F16162) ---

Visa de long séjour (séjour de plus de 3 mois à 1 an) | Service Public
Ce site utilise des cookies permettant de visualiser des contenus et d'améliorer le fonctionnement grâce aux statistiques de navigation. Si vous cliquez sur « Accepter », la Dila (éditeur du site Service-Public.fr) et ses partenaires déposeront ces cookies sur votre terminal lors de votre navigation. Si vous cliquez sur « Refuser », ces cookies ne seront pas déposés. Votre choix est conservé pendant 6 mois et vous pouvez être informé et modifier vos préférences à tout moment sur la page « Gérer les cookies »
Accepter
Refuser
Personnaliser
Javascript est desactivé dans votre navigateur.
Contenu
Menu
Rech

2. **RAG**

In [ ]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.openai import OpenAI


In [ ]:
# Définir le modèle d'embedding
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Définir la mémoire
memory = ChatMemoryBuffer.from_defaults(token_limit=1000)

# Définir le LLM
llm = MistralAI(
    api_key=mistral_key_api,
    temperature=0.7,
    additional_kwargs={"stream": False}
    )

# Construire l'index vectoriel
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Créer le moteur de requêtes avec le LLM et la mémoire
query_engine = index.as_query_engine(llm=llm, memory=memory)



INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [ ]:
# Test de requêtes
print(query_engine.query("Quels documents sont nécessaires pour un visa de court séjour ?"))
print(query_engine.query("Quelles sont les conditions pour déposer une demande ?"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Pour un visa de court séjour, les documents nécessaires varient selon votre situation professionnelle ou personnelle. Voici les pièces justificatives requises selon différents profils :

### **1. Pour les personnes exerçant une profession ou une activité spécifique :**
- **Membres d’institutions officielles** (Cour royale, gouvernement, Parlement, etc.) : une **note verbale**, une **attestation de fonction** ou un autre document officiel.
- **Hauts fonctionnaires de l’État marocain** : une **attestation de fonction** délivrée par l’organisme compétent.
- **Recteurs d’université ou doyens** : une **attestation de fonction** délivrée par l’université concernée.
- **Personnel diplomatique ou consulaire** (Union européenne, ambassades, consulats) : une **attestation de fonction** délivrée par l’employeur.
- **Conjoint ou enfant mineur à charge** des personnes citées ci-dessus : une **preuve du lien de parenté**.
- **Bénéficiaires d’une bourse** (Erasmus+, Horizon 2020, etc.) : une **lettre

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Pour déposer une demande de visa "Vacances-travail" (PVT), voici les conditions principales à respecter selon votre pays d'origine :

- **Âge** :
  - **18 à 35 ans** : si vous êtes ressortissant d'**Australie** ou du **Canada**.
  - **18 à 30 ans** : si vous venez du **Brésil**, du **Chili**, de **Chine (Hong-Kong)**, de la **Colombie**, de la **Corée du Sud**, de l'**Équateur**, du **Japon**, du **Mexique**, de la **Nouvelle-Zélande** ou du **Pérou**.

- **Durée du séjour** :
  - La plupart des visas ont une durée maximale de **12 mois**, sans possibilité de prolongation ni de changement de statut (sauf exceptions pour le Canada et sous conditions spécifiques).
  - Pour le **Canada**, la durée peut aller jusqu’à **24 mois**, voire **36 mois** dans certains cas (études universitaires dans le cadre d’un accord interuniversitaire ou stage pratique pour étudiants/jeunes en formation).

- **Restrictions** :
  - Ce visa ne permet pas de demander une carte de séjour pour un autre motif, sauf

In [ ]:
print(query_engine.query("combien coûte un billet d'avion Maroc -  France ?"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Je ne peux pas fournir d'informations sur le coût d'un billet d'avion entre le Maroc et la France.


In [ ]:
# demander mistral à dire bonjour
print(query_engine.query("Bonjour"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Bonjour ! Comment puis-je vous aider concernant un visa de long séjour en France ?


# Problème de clé OpenAI pour continuer avec Agent

3. **Agent AI**

In [ ]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import FunctionTool


# Créer un outil basé sur  query_engine
visa_query_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="Visa_Bot_IA",
    description="Outil pour répondre aux questions sur les documents et démarches de visa de court séjour."
)



4. **Tools**

In [ ]:
from llama_index.core.tools import FunctionTool

def check_documents(user_country: str) -> str:
    if user_country.lower() == "maroc":
        return "Passeport, formulaire rempli, photo, justificatif de ressources, attestation de logement."
    return f"Les documents pour {user_country} ne sont pas encore disponibles."

def find_embassy(city: str) -> str:
    return f"L'ambassade à {city} est située au centre-ville, rue des Consulats."

def check_visa_delay(country: str) -> str:
    return f"Le délai moyen pour un visa de court séjour pour {country} est de 15 jours."

tools = [
    visa_query_tool,
    FunctionTool.from_defaults(fn=check_documents, name="check_documents", description="Documents requis"),
    FunctionTool.from_defaults(fn=find_embassy, name="find_embassy", description="Trouver l'ambassade"),
    FunctionTool.from_defaults(fn=check_visa_delay, name="check_visa_delay", description="Délais de visa")
]


5. **Agent avec une personnalité**

In [ ]:
from llama_index.core.agent import ReActAgent

# Créer un agent avec une personnalité
agent = ReActAgent(
    tools=tools,
    llm=llm,
    system_prompt="Tu es un assistant expert en visas de court séjour. Réponds toujours clairement.",
    verbose=True
)


# Il faut vraiment une clé openai pour pouvoir utiliser tools avec agent

In [ ]:
print(query_engine.query("Bonjour"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Bonjour ! Comment puis-je vous aider concernant un visa de long séjour en France ?


In [ ]:
print(query_engine.query("Comment proceder pour un visa de long séjour ?"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Pour obtenir un **visa de long séjour**, voici les étapes clés à suivre :

1. **Déterminer le type de visa** en fonction de votre situation (études, travail, raisons familiales, etc.). Le **Visa de Long Séjour valant Titre de Séjour (VLS-TS)** est le plus courant, valable de **4 à 12 mois**, et dispense d’une demande de carte de séjour en préfecture à l’arrivée en France.

2. **Faire la demande** auprès des **autorités consulaires françaises** (ambassade ou consulat) dans votre pays de résidence. Les motifs les plus fréquents sont les études, le travail ou le regroupement familial.

3. **Respecter les conditions** :
   - Ne pas dépasser **90 jours de séjour dans l’espace Schengen** pendant la validité du visa.
   - Si vous êtes majeur, un visa de long séjour est obligatoire, même si votre nationalité vous exonère habituellement d’un visa de court séjour.

4. **Vérifier les cas de refus** :
   - Si vous avez fait l’objet d’une **obligation de quitter le territoire français** (sans preuv

In [ ]:
print(query_engine.query("j'ai visa, comment proceder le voyage ?"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Pour préparer votre voyage au Maroc avec un visa de court séjour, voici les étapes et documents clés à vérifier avant votre départ :

1. **Vérifiez vos documents de voyage** :
   - Assurez-vous d’avoir votre **passeport valide** (avec le visa apposé) et une **copie numérique** en cas de perte.
   - Confirmez votre **réservation de billet aller-retour** (avion, bus ou bateau), car elle peut être demandée à l’embarquement ou à l’arrivée.

2. **Préparez vos preuves d’hébergement** :
   - Si vous séjournez à l’hôtel, imprimez ou sauvegardez la **confirmation de réservation**.
   - Si vous êtes hébergé par un particulier ou une entreprise, munissez-vous de la **lettre d’invitation** (ou formulaire officiel si requis) et des preuves de prise en charge financière, le cas échéant.
   - Si vous louez un logement, apportez le **contrat de location** ou un justificatif de propriété.

3. **Justifiez vos moyens financiers** :
   - Emportez les **relevés bancaires** des trois derniers mois (si vous 

In [ ]:
print(query_engine.query("Comment contacter l'ambassade Maroc zn france?"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Pour obtenir des informations générales sur les démarches administratives en France, vous pouvez contacter **Allô Service Public** aux horaires suivants :

- **Lundi** : 08h30 – 17h30
- **Mardi** : 08h30 – 12h15
- **Mercredi** : 08h30 – 12h15
- **Jeudi** : 08h30 – 17h30
- **Vendredi** : 13h00 – 16h15

Vous avez également la possibilité de **demander un rappel téléphonique** via le lien dédié sur le site officiel.

*Note* : Ce service ne peut pas accéder aux dossiers personnels ni renseigner sur leur état d’avancement. Pour des questions spécifiques concernant l’ambassade du Maroc en France, il est recommandé de consulter directement leur site officiel ou leurs coordonnées.


In [ ]:
print(query_engine.query("combien coûte un visa de long séjour au Maroc ?"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Je n'ai pas d'information sur le coût d'un visa de long séjour pour le Maroc dans les détails fournis. Les éléments disponibles concernent uniquement les visas de long séjour pour la France.


In [ ]:
print(query_engine.query("combien coûte un billet d'avion Maroc -  France ?"))

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Je n'ai pas d'information concernant le coût d'un billet d'avion entre le Maroc et la France.
